In [2]:
import os
import torch
import torch.nn as nn
import joblib
import numpy as np
import control as ct
import matplotlib.pyplot as plt

In [3]:
# Глобальные параметры системы
beta = 24.68368653
Tm = 0.25
Te = 0.125
Tfc = 0.001
kfc = 5.0
k_fb = 0.3
b = 2.08333
h0 = 0.675
x0 = 1.18272615
Tp = 0.104

Kp_cl = 15.47
Ki_cl = 12.26

In [4]:
# Нейросетевая модель PI-регулятора
class PINet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )
    def forward(self, x):
        return self.net(x)

In [5]:
# Загрузка модели и масштабаторов
model = PINet()
model.load_state_dict(torch.load("ML-neuro-pi2.pt"))
model.eval()

scaler_X = joblib.load("scaler_X.pkl")
scaler_Y = joblib.load("scaler_Y.pkl")

def predict_kp_ki(x_state):
    x_input = scaler_X.transform([x_state])
    x_tensor = torch.tensor(x_input, dtype=torch.float32)
    with torch.no_grad():
        y_pred = model(x_tensor).numpy()
    kp_ki = scaler_Y.inverse_transform(y_pred)
    return kp_ki[0]

C:\Users\USER\AppData\Local\Temp\ipykernel_8652\4242249903.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("ML-neuro-pi2.pt"))


In [6]:
# Глобальные списки для истории
kp_history = []
ki_history = []
kp_hybrid_history = []
ki_hybrid_history = []
time_history = []

In [7]:
# ОПИСАНИЕ СИСТЕМ

def pump_neuro_pi_system(t, x, u, params=None):
    P = u[0]
    x1, x2, x3, x4, x5 = x

    Kp_ad, Ki_ad = predict_kp_ki([x1, x2, x3, x4, x5])
    kp_history.append(Kp_ad)
    ki_history.append(Ki_ad)
    time_history.append(t)

    dx1_dt = (1 / (beta * Tm)) * (x2 - x1)
    dx2_dt = (beta / Te) * x3 - (beta / Te) * x1 - (1 / Te) * x2
    dx3_dt = (kfc / Tfc) * x4 - (1 / Tfc) * x3
    dx5_dt = ((b / Tp) + ((2 * h0 * x0) / Tp)) * x1 - (1 / Tp) * x5
    dx5_dt = np.clip(dx5_dt, -10, 10)
    x5 = np.clip(x5, 0.7, 5.7)
    dx4_dt = P - Kp_ad * k_fb * dx1_dt - Ki_ad * k_fb * x1
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt]

def pump_classic_pi_system(t, x, u, params=None):
    P = u[0]
    x1, x2, x3, x4, x5 = x

    dx1_dt = (1 / (beta * Tm)) * (x2 - x1)
    dx2_dt = (beta / Te) * x3 - (beta / Te) * x1 - (1 / Te) * x2
    dx3_dt = (kfc / Tfc) * x4 - (1 / Tfc) * x3
    dx5_dt = ((b / Tp) + ((2 * h0 * x0) / Tp)) * x1 - (1 / Tp) * x5
    dx5_dt = np.clip(dx5_dt, -10, 10)
    x5 = np.clip(x5, 0.7, 5.7)
    dx4_dt = P - Kp_cl * k_fb * dx1_dt - Ki_cl * k_fb * x1
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt]

def pump_hybrid_pi_system(t, x, u, params=None):
    P = u[0]
    x1, x2, x3, x4, x5 = x

    Kp_ad, Ki_ad = predict_kp_ki([x1, x2, x3, x4, x5])
    Kp_h = (Kp_ad + Kp_cl) / 2
    Ki_h = (Ki_ad + Ki_cl) / 2

    kp_hybrid_history.append(Kp_h)
    ki_hybrid_history.append(Ki_h)

    dx1_dt = (1 / (beta * Tm)) * (x2 - x1)
    dx2_dt = (beta / Te) * x3 - (beta / Te) * x1 - (1 / Te) * x2
    dx3_dt = (kfc / Tfc) * x4 - (1 / Tfc) * x3
    dx5_dt = ((b / Tp) + ((2 * h0 * x0) / Tp)) * x1 - (1 / Tp) * x5
    dx5_dt = np.clip(dx5_dt, -10, 10)
    x5 = np.clip(x5, 0.7, 5.7)
    dx4_dt = P - Kp_h * k_fb * dx1_dt - Ki_h * k_fb * x1
    return [dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt]

def spump_output(t, x, *args):
    return x

In [8]:
# СИСТЕМЫ
systems = {
    "Нейросетевой PI": ct.NonlinearIOSystem(pump_neuro_pi_system, spump_output, states=5, name='spump', inputs=('P',), outputs=('x1', 'x2', 'x3', 'x4', 'x5')),
    "Классический PI": ct.NonlinearIOSystem(pump_classic_pi_system, spump_output, states=5, name='classic_spump', inputs=('P',), outputs=('x1', 'x2', 'x3', 'x4', 'x5')),
    "Гибридный PI": ct.NonlinearIOSystem(pump_hybrid_pi_system, spump_output, states=5, name='hybrid_spump', inputs=('P',), outputs=('x1', 'x2', 'x3', 'x4', 'x5')),
}

In [9]:
# СИМУЛЯЦИЯ

T = np.linspace(0, 30, 500)
U = (5.0 * (1 - np.exp(-T / 3))).reshape(1, -1)
X0 = np.zeros(5)

results = {}
for name, system in systems.items():
    if name == "Нейросетевой PI":
        kp_history.clear()
        ki_history.clear()
        time_history.clear()
    if name == "Гибридный PI":
        kp_hybrid_history.clear()
        ki_hybrid_history.clear()

    t, y = ct.input_output_response(system, T, U, X0, solve_ivp_kwargs={'method': 'Radau'})
    results[name] = (t, y)

In [10]:
# ВИЗУАЛИЗАЦИЯ

os.makedirs("plots", exist_ok=True)

state_labels = ['x1 (Δω)', 'x2 (ΔM)', 'x3 (Δω₀)', 'x4 (ΔU_РД)']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red']

for name, (t, y) in results.items():
    plt.figure(figsize=(7, 3))
    for i in range(4):
        plt.plot(t, y[i], label=state_labels[i], color=colors[i])
    plt.title(f'{name}: переменные состояния x1–x4')
    plt.xlabel("Время")
    plt.ylabel("Значения переменных")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"plots/{name}_states.png")
    plt.close()

for name, (t, y) in results.items():
    plt.figure(figsize=(7, 3))
    plt.plot(t, y[4], label='x5 (давление)', color='tab:blue')
    plt.title(f'{name}: поведение давления x5')
    plt.xlabel("Время")
    plt.ylabel("Давление x5")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"plots/{name}_pressure.png")
    plt.close()

if time_history:
    plt.figure(figsize=(8, 3))
    plt.plot(time_history, kp_history, label='Kp (нейросетевой)', color='tab:blue')
    plt.plot(time_history, kp_hybrid_history, label='Kp (гибридный)', color='tab:green')
    plt.xlabel("Время")
    plt.ylabel("Kp")
    plt.title("Сравнение Kp: нейросетевой vs гибридный PI")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("plots/Kp_comparison.png")
    plt.close()

    plt.figure(figsize=(8, 3))
    plt.plot(time_history, ki_history, label='Ki (нейросетевой)', color='tab:orange')
    plt.plot(time_history, ki_hybrid_history, label='Ki (гибридный)', color='tab:red')
    plt.xlabel("Время")
    plt.ylabel("Ki")
    plt.title("Сравнение Ki: нейросетевой vs гибридный PI")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("plots/Ki_comparison.png")
    plt.close()

print("Симуляция завершена. Графики сохранены в папку 'plots/'")

Симуляция завершена. Графики сохранены в папку 'plots/'
